In [13]:
import keras_tuner as kt
import tensorflow as tf
from tensorflow.keras.layers import  Flatten , Dense
from tensorflow.keras.models import Sequential


In [5]:
(x_train , y_train) , (x_test , y_test) = tf.keras.datasets.mnist.load_data()

11490434/11490434 [==============================] - 2s 0us/step


In [6]:
(x_train.shape , y_train.shape) , (x_test.shape , y_test.shape)

(((60000, 28, 28), (60000,)), ((10000, 28, 28), (10000,)))

In [7]:
x_train = x_train/255.0
x_test = x_test/255.0

In [24]:
def model_builder(hp):
    model= Sequential()
    model.add(tf.keras.layers.Flatten(input_shape = (28,28)))
    
    
    hp_activation = hp.Choice('activation',values = ['relu','tanh'])
    
    hp_layer_1 = hp.Int('layer_1',min_value = 1 , max_value = 1000 , step = 100)
    hp_layer_2 = hp.Int('layer_2',min_value = 1 , max_value = 1000 , step = 100)
    hp_learning_rate = hp.Choice('learning_rate' , values = [1e-2,1e-3,1e-4])

    model.add(tf.keras.layers.Dense(units = hp_layer_1 , activation = hp_activation ))
    model.add(tf.keras.layers.Dense(units = hp_layer_2 , activation = hp_activation ))
    model.add(tf.keras.layers.Dense(units = 10 , activation = 'softmax' ))
    
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate =hp_learning_rate ),
                 loss = tf.keras.losses.SparseCategoricalCrossentropy(),
                 metrics = ['accuracy'])
    model.summary()
    return model

In [25]:
tuner = kt.Hyperband(model_builder,
                    objective = 'val_accuracy',
                    max_epochs = 10,
                    factor = 3,
                    directory = 'dir',
                    project_name = 'x')

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (None, 784)               0         
                                                                 
 dense_9 (Dense)             (None, 1)                 785       
                                                                 
 dense_10 (Dense)            (None, 1)                 2         
                                                                 
 dense_11 (Dense)            (None, 10)                20        
                                                                 
Total params: 807 (3.15 KB)
Trainable params: 807 (3.15 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [26]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss' , patience = 3)


In [27]:
tuner.search(x_train , y_train , epochs = 50  , validation_split = 0.2 , callbacks = [stop_early])

Trial 30 Complete [00h 03m 21s]
val_accuracy: 0.9733333587646484

Best val_accuracy So Far: 0.9775000214576721
Total elapsed time: 00h 21m 45s


In [28]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [29]:
best_hps

In [30]:
model = tuner.hypermodel.build(best_hps)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 901)               707285    
                                                                 
 dense_4 (Dense)             (None, 501)               451902    
                                                                 
 dense_5 (Dense)             (None, 10)                5020      
                                                                 
Total params: 1164207 (4.44 MB)
Trainable params: 1164207 (4.44 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [32]:
history = model.fit(x_train ,y_train , epochs = 50 , validation_split = 0.2 , callbacks = [stop_early])

Epoch 1/50
1500/1500 [==============================] - 23s 15ms/step - loss: 0.3359 - accuracy: 0.9105 - val_loss: 0.1689 - val_accuracy: 0.9523
Epoch 2/50
1500/1500 [==============================] - 21s 14ms/step - loss: 0.1386 - accuracy: 0.9602 - val_loss: 0.1204 - val_accuracy: 0.9659
Epoch 3/50
1500/1500 [==============================] - 18s 12ms/step - loss: 0.0922 - accuracy: 0.9734 - val_loss: 0.1048 - val_accuracy: 0.9693
Epoch 4/50
1500/1500 [==============================] - 18s 12ms/step - loss: 0.0657 - accuracy: 0.9812 - val_loss: 0.0890 - val_accuracy: 0.9735
Epoch 5/50
1500/1500 [==============================] - 18s 12ms/step - loss: 0.0476 - accuracy: 0.9862 - val_loss: 0.0820 - val_accuracy: 0.9749
Epoch 6/50
1500/1500 [==============================] - 18s 12ms/step - loss: 0.0350 - accuracy: 0.9902 - val_loss: 0.0769 - val_accuracy: 0.9768
Epoch 7/50
1500/1500 [==============================] - 18s 12ms/step - loss: 0.0261 - accuracy: 0.9930 - val_loss: 0.0771 -

In [33]:
history.history

{'loss': [0.3358514606952667,
  0.13858941197395325,
  0.09223749488592148,
  0.06572352349758148,
  0.047619327902793884,
  0.034969866275787354,
  0.026092959567904472,
  0.018697567284107208,
  0.014695745892822742,
  0.01051856018602848,
  0.009236058220267296,
  0.006446673069149256],
 'accuracy': [0.9104999899864197,
  0.9601874947547913,
  0.973395824432373,
  0.9811875224113464,
  0.9862083196640015,
  0.9901875257492065,
  0.9930416941642761,
  0.995354175567627,
  0.9962499737739563,
  0.9976041913032532,
  0.9975416660308838,
  0.9986458420753479],
 'val_loss': [0.1688825935125351,
  0.12042882293462753,
  0.10482382029294968,
  0.08901070058345795,
  0.08198689669370651,
  0.0768558531999588,
  0.07714326679706573,
  0.07751260697841644,
  0.07558439671993256,
  0.08773920685052872,
  0.08518388122320175,
  0.08348570764064789],
 'val_accuracy': [0.9522500038146973,
  0.9659166932106018,
  0.9693333506584167,
  0.9735000133514404,
  0.9749166369438171,
  0.9768333435058594,